In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
import re
#from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout
from keras import backend as K
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D
from sklearn.datasets import load_iris

2024-07-30 23:53:17.957536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 23:53:17.988478: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 23:53:17.997729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 23:53:18.022885: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
directory = "/scratch/braines/Dataset/CCMT-Dataset-Augmented/train_Data/Cashew"

In [11]:
def get_label(directory: str):
        file_list = []
        for x in os.walk(directory, topdown=True):
                if x[2] != []:
                        for y in x[2]:
                                label = str("".join(re.findall("[a-zA-Z]",x[0].split("/", -1)[-1])))
                                temp = (y, label)
                                file_list.append(temp)
        file_list.sort()
        label_names = []
        values = []
        for x in file_list:
                label_names.append(x[1])
                if x[1] not in values:
                        values.append(x[1])
        mapping = {}
        count = 0
        for x in values:
                mapping[x] = count
                count += 1
        labels = []
        for x in label_names:
                labels.append(mapping[x])

        return labels, mapping


In [12]:
label_list, label_mapping = get_label(directory)

In [13]:
label_list

[0,
 1,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 3,
 0,
 1,
 4,
 0,
 1,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 0,
 1,
 4,
 2,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 3,
 0,
 1,
 4,
 0,
 1,
 4,
 3,
 0,
 1,
 4,


In [20]:
z = keras.utils.image_dataset_from_directory(directory, labels=label_list, verbose=True, \
                                             label_mode="categorical", pad_to_aspect_ratio=True, validation_split=0.1 \
                                            , subset="both", seed=65)

Found 18910 files belonging to 5 classes.
Using 17019 files for training.
Using 1891 files for validation.


In [21]:
def vgg16():
    model = Sequential()

    model.add(Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), padding="same",activation="relu"))

[<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 5), dtype=tf.float32, name=None))>]